# Exercise: Surface Diffusion via Kinetic Monte Carlo

In this notebook, we explore surface diffusion phenomena using a lattice gas model on a 2D honeycomb lattice, solved via Kinetic Monte Carlo (KMC). This approach allows us to model how adsorbed atoms (adsorbates) migrate across a surface over time by hopping between discrete lattice sites.


### What is the lattice gas model?

The lattice gas model simplifies surface diffusion by representing the surface as a regular grid (lattice) of discrete sites. Each site can either be occupied (by an adsorbate) or vacant, and particles hop between neighboring sites.

In our framework, these hops are governed by probabilistic rules derived from a simplified energy model:

* We do not compute atomistic energy barriers from first principles.
* Instead, we assign fixed interaction energies (e.g., for neighboring particles or site preferences), and use these to compute relative jump probabilities.
* These probabilities follow a Metropolis-like rule:

  $$
  P_{\text{jump}} \propto 
  \begin{cases}
  \exp\left(-\frac{\Delta E}{kT}\right) & \text{if } \Delta E > 0 \\\\
  1 & \text{otherwise}
  \end{cases}
  $$

  where $\Delta E$ is the energy change associated with the hop.


### How is time handled in Kinetic Monte Carlo?

There are two main interpretations of time in KMC simulations, depending on the level of physical input:

#### Atomistic KMC: absolute rates and real time

In atomistic KMC, one often computes jump rates from barriers (e.g., using transition state theory):

$$
\Gamma = \nu \cdot e^{-E_a / kT}
$$

These rates have units of 1/s, so the algorithm accumulates a physically meaningful time via:

$$
\Delta t = \frac{1}{\sum_i \Gamma_i}
$$

In this case, KMC simulates actual time evolution.

#### Lattice MC (**like in this notebook**): relative rates and dimensionless time

* In lattice-based models, we often want to explore generic behavior without specifying exact chemistry.
* The simulations in this notebook are performed using a *rejection-free kinetic Monte Carlo (KMC) scheme*. This means:
    * All possible particle hops are assigned a relative rate, which may depend on local site occupancy or interaction energies.
    * A hop is chosen with probability proportional to its rate.
    * The simulation clock is advanced using:

  $$
  \Delta t = \frac{1}{\sum_i r_i}
  $$

In our notebook, we typically use unitless rates, so the simulated time remains dimensionless, but still faithfully tracks the relative pace of diffusion under different assumptions.

> **Note**: This approach is designed for efficiency and pedagogical clarity. We do not aim to perform large-scale simulations with statistically converged transport properties. That would require much longer runs and careful parameterization of the potential energy landscape, which is absolutely possible, just not the goal here.

<div style="text-align: center;">
    <img src="https://www.frontiersin.org/files/Articles/434159/fchem-07-00202-HTML/image_m/fchem-07-00202-g002.jpg", width=600>
</div>

In [ ]:
! pip install lattice_mc
import lattice_mc
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context("talk")
import pandas as pd
from math import fabs
from tqdm.notebook import tqdm
from scipy.interpolate import interp1d
import copy
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# define temperature scale
kT = lattice_mc.global_vars.kT # default setting is T=298 K

# Task (a) Non-interacting particles

The key assumption in this task is that particles are non-interacting: they do not feel any energetic attraction or repulsion from other particles. 

However, volume exclusion is enforced: a particle cannot hop into an already occupied site.

We simulate atomic diffusion on a honeycomb lattice with $48$ sites.

Modify the following variables to control the simulation:

- `n_steps`:
Number of MC steps.
Set to $10000$ to allow atoms enough time to explore the lattice.

- `n_samples`:
Number of repeated simulations for each atom count.
Set to $100$ to get statistically meaningful averages.

- `n_atoms_list`:
A list of atom counts you want to simulate.
Use $1, 2, 4$.

Run the simulations and execute the cell that plots the mean squared displacement as a function of time.

In [ ]:
# JUST A FIGURE
np.random.seed(23)
lattice = lattice_mc.init_lattice.honeycomb_lattice(4, 2, 1)
lattice.populate_sites(5)

def plot_lattice(lattice, ax=None):
    # Extract positions
    occupied_positions = np.array([site.r[:2] for site in lattice.occupied_sites()])
    vacant_positions = np.array([site.r[:2] for site in lattice.vacant_sites()])

    # Use provided Axes or create new one
    if ax is None:
        fig, ax = plt.subplots(figsize=(5, 5))

    ax.scatter(*vacant_positions.T, s=200, facecolors='none', edgecolors='black')
    ax.scatter(*occupied_positions.T, marker="H", s=300, color='orangered')

    ax.set_aspect('equal')
    ax.axis('off')
    ax.margins(0.2)

    return ax  # Optionally return ax if needed

fig = plot_lattice(lattice)

In [ ]:
# COMPLETE AND EXECUTE THIS CELL
# === Parameters ===
n_steps = 10000            # Number of steps (jumps) per simulation
n_samples = 100             # Number of simulation samples per atom count
n_atoms_list = [1, 2, 4]   # List of atom counts to simulate
# ========================================

def simulation_wrapper_msd(n_atoms, _):
    a, b = 4, 3
    spacing = 1.0

    s = lattice_mc.Simulation()
    s.lattice = lattice_mc.init_lattice.honeycomb_lattice(a, b, spacing)
    s.set_number_of_atoms(n_atoms)
    s.set_number_of_jumps(n_steps)

    times = []
    msd = []

    s.lattice.reset()
    for _ in range(n_steps):
        s.lattice.jump()
        times.append(s.lattice.time)
        msd.append(s.atoms.sum_dr_squared() / n_atoms)

    return np.array(times), np.array(msd)

def run_msd_ensemble(n_atoms, nsamples=25, n_time_points=200):
    results = []
    for i in tqdm(range(nsamples), desc=f"{n_atoms} atoms"):
        results.append(simulation_wrapper_msd(n_atoms, i))

    # Find minimum final time across all samples
    min_t_max = min(times[-1] for times, _ in results)
    t_common = np.linspace(0, min_t_max, n_time_points)

    records = []
    for sample_id, (times, msd) in enumerate(results):
        f_interp = interp1d(times, msd, kind='linear', bounds_error=False, fill_value='extrapolate')
        msd_interp = f_interp(t_common)

        for t, v in zip(t_common, msd_interp):
            records.append({
                "sample": sample_id,
                "time": t,
                "msd": v,
                "n_atoms": n_atoms
            })

    return pd.DataFrame(records)

# === Run simulations and collect all data ===
df_all = pd.concat(
    [run_msd_ensemble(n_atoms=n, nsamples=n_samples) for n in n_atoms_list],
    ignore_index=True
)

In [ ]:
# EXECUTE THIS CELL
plt.figure(figsize=(6, 5))

sns.lineplot(data=df_all, x="time", y="msd", hue="n_atoms", errorbar="sd", palette="Set2")

plt.xlabel("Time")
plt.ylabel("MSD")

#plt.xscale("log")
plt.grid(alpha=0.5)

plt.legend(title=r"$n_{\text{atoms}}$", fontsize=14)
plt.tight_layout()
plt.show()

Do you notice anything?

When performing kinetic Monte Carlo (kMC) simulations at different coverages (or concentrations), what practical considerations should be made to ensure that the results are comparable across coverages? 

Specifically, does running the same number of kMC events correspond to the same physical time scale at each coverage, or do differences in event rates and dynamics require adjustment?

Discuss.

## Task (b) Transport properties as a function of coverage

The lattice has a fixed number of sites ($48$).

You will simulate diffusion of particles for different coverages $\Theta$, defined by the number of particles on the lattice.
This setup allows you to simulate how transport properties vary with the number of particles, even without any inter-particle interaction, just due to crowding and excluded volume effects.


Define a list of particle counts corresponding to different coverages:

* $1$ particle (dilute limit)

* $1$ vacancy

* $25\%$ coverage

* $50\%$ coverage

* $75\%$ coverage

The simulation returns a summary of key transport quantities: tracer diffusivity, $D$, and correlation factor, $f$.

Discuss the dependence of $D$ and $f$ on $\Theta$. 

> **What is the correlation factor?**

In a random walk, the mean squared displacement (MSD) is defined as:
$
\text{MSD}_r = \sum \langle x_i^2 \rangle.
$

 If the jumps are correlated (i.e., not random), deviations from ideality are packaged in the *correlation factor*:
$
\text{MSD} = f \cdot \text{MSD}_r.
$

 For a pure random walk, $f = 1$. 

 Although $0<f<1$, you might observe unphysical values due to finite-size effects and poor statistics.

In [ ]:
# EXECUTE THIS CELL
def honeycomb_lattice_correlation(n_atoms):
    a, b = 4, 3
    spacing = 1.0
    n_steps = 10000

    s = lattice_mc.Simulation()
    s.lattice = lattice_mc.init_lattice.honeycomb_lattice(a, b, spacing)
    s.set_number_of_atoms(n_atoms)
    s.set_number_of_jumps(n_steps)
    s.setup_lookup_table()
    s.run()

    return s.tracer_diffusion_coefficient, s.tracer_correlation

# === Function: run multiple samples for each atom count ===
def run_simulations(n_atoms_list, simulation_function, n_samples=100):
    results = []
    for n_atoms in n_atoms_list:
        samples = np.array([
            simulation_function(n_atoms)
            for _ in tqdm(range(n_samples), desc=f"{n_atoms} atoms")
        ])
        mean_values = np.mean(samples, axis=0)
        results.append([n_atoms, *mean_values])
    return results

In [ ]:
# COMPLETE AND EXECUTE THIS CELL
# === Parameters ===
n_atoms_list = [1, 12, 24, 36, 47]  
n_samples = 100

# === Run simulations and collect results ===
results = run_simulations(n_atoms_list, honeycomb_lattice_correlation, n_samples=n_samples)
data_non = pd.DataFrame(results, columns=["n", "D*", "f"])
data_non

In [ ]:
# EXECUTE THIS CELL
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(5, 8))

# First plot: Tracer Diffusivity
ax1.plot(data_non['n'] / len(s.lattice.sites), data_non['D*'], 'o-', c='midnightblue')
ax1.set_yscale('log')
ax1.grid(alpha=0.5)
ax1.set_ylabel("Tracer Diffusivity")

# Second plot: Correlation Factor
ax2.plot(data_non['n'] / len(s.lattice.sites), data_non['f'], 'o-', c='midnightblue')
ax2.set_ylim(0.0, 1.2)
ax2.axhline(1/3, color='tomato', ls='--', label=f"exact result,\nsingle vacancy")
ax2.grid(alpha=0.5)
ax2.legend(fontsize=14)
ax2.set_xlabel(r"Coverage $\Theta$")
ax2.set_ylabel("Correlation factor")

plt.tight_layout()
plt.show()

<details>
<summary>Solution</summary>
In kinetic Monte Carlo simulations, the total number of events does not necessarily translate to the same physical time at different coverages, because:

* The event rate can vary significantly with coverage due to changes in available sites and interaction effects.
* The attempt frequency and success rate of moves may be lower at higher coverage, affecting the simulation time increment per event.

Thus, to ensure comparability one should consider normalizing time using cumulative time from the kMC clock (i.e., physical time, not number of events).
</details>

# Task (c) Interacting Particles

In the case of non-interacting particles, all allowed jumps are equally probable. However, when interactions are introduced, the probability of each jump becomes dependent on the change in the system's total energy caused by that jump.

In this task, we implement nearest-neighbor repulsion. The total energy of a configuration is calculated by summing over each occupied site $j$, where the contribution of each site depends on the number of its occupied nearest neighbors:

$$
E = \sum_j n_j^{\text{nn}} E_{\text{nn}}
$$

* $n_j^{\text{nn}}$ is the number of occupied nearest-neighbor sites for occupied site $j$.
* $E_{\text{nn}}$ is the interaction energy per neighboring pair (positive for repulsion).


You will perform simulations for different coverages sweeping over various values of the interaction energy $E_{\text{nn}} = 0.0, 0.2, 0.4$.

Set the number of samples to $50$ to speed up the simulations.

Analyze the properties returned by the simulations:

  * The correlation factor,
  * The order parameter $|n_A - n_B| / N$ that quantifies how particles preferentially occupy the two different sublattices $A$ and $B$.

Do you observe any unexpected behavior?
What physical mechanisms might be responsible for this behavior?
While direct visualization of trajectories is not available, try to infer the dynamics based on your physical intuition and understanding of the model.

In [ ]:
# JUST A FIGURE
np.random.seed(23)
lattice = lattice_mc.init_lattice.honeycomb_lattice(4, 2, 1)
lattice.populate_sites(5)


def plot_lattice(lattice, ax=None):
    # Extract positions
    occupied_positions = np.array([site.r[:2] for site in lattice.occupied_sites()])
    vacant_positions = np.array([site.r[:2] for site in lattice.vacant_sites()])

    # Use provided Axes or create new one
    if ax is None:
        fig, ax = plt.subplots(figsize=(5, 5))

    ax.scatter(*vacant_positions.T, s=200, facecolors='none', edgecolors='black')
    ax.scatter(*occupied_positions.T, marker="H", s=300, color='orangered')
    ax.scatter(*occupied_positions.T, s=1000, color='orangered', alpha=0.5)

    ax.set_aspect('equal')
    ax.axis('off')
    ax.margins(0.2)

    return ax  # Optionally return ax if needed

fig = plot_lattice(lattice)

In [ ]:
# EXECUTE THIS CELL
def honeycomb_lattice_correlation_nn(n_atoms, nn_repulsion):
    a, b = 4, 3
    spacing = 1.0
    n_jumps = 10000
    n_equilibration_jumps = 100

    s = lattice_mc.Simulation()
    s.lattice = lattice_mc.init_lattice.honeycomb_lattice(
        a, b, spacing, alternating_sites=True
    )
    s.set_number_of_atoms(n_atoms)
    s.set_number_of_jumps(n_jumps)
    s.set_number_of_equilibration_jumps(n_equilibration_jumps)
    s.set_nn_energy(nn_repulsion * kT)
    s.setup_lookup_table()
    s.run()

    # Calculate order parameter from A/B site occupation imbalance
    order_parameter = fabs(
        s.average_site_occupations['A'] - s.average_site_occupations['B']
    ) / 24  # Max imbalance for full coverage on one sublattice

    return s.tracer_correlation, order_parameter


# === Function: Run multiple simulations for a range of atoms and interaction energies ===
def run_simulations(n_atoms_list, nn_energy_list, simulation_function, n_samples=50):
    results = []
    for n_atoms in n_atoms_list:
        print(f"Running simulations for n_atoms = {n_atoms}")
        for nn_energy in nn_energy_list:
            samples = np.array([
                simulation_function(n_atoms, nn_energy)
                for _ in tqdm(range(n_samples), desc=f"  nn_energy = {nn_energy}")
            ])
            mean_values = np.mean(samples, axis=0)
            results.append([n_atoms, nn_energy, *mean_values])
    return results

In [ ]:
# COMPLETE AND EXECUTE THIS CELL
# === Parameters ===
n_atoms_list = [1, 12, 24, 36, 47]
nn_energy_list = [0.0, 2.0, 4.0]
n_samples = 50

# === Run simulations and collect results ===
results_nn = run_simulations(n_atoms_list, nn_energy_list, honeycomb_lattice_correlation_nn, n_samples=n_samples)
data_nn = pd.DataFrame(results_nn, columns=['n_atoms', 'E_nn', 'f', 'order_parameter'])
data_nn

In [ ]:
# EXECUTE THIS CELL
palette = sns.color_palette("colorblind", n_colors=len(nn_energy))

# Create subplots
fig, axes = plt.subplots(2, 1, figsize=(6, 8), sharex=True)
titles = [
    "Correlation Factor",
    "Site Preference Asymmetry"
]
ylabels = [r"$f$", r"$|n_A - n_B| / N$"]

# Plot each quantity
for idx, nn in enumerate(nn_energy):
    nn_data = data_nn[data_nn.E_nn == nn]
    axes[0].plot(nn_data.n_atoms/48, nn_data.f, 'o-', label=rf"$E_{{nn}} = {nn}$", color=palette[idx])
    axes[1].plot(nn_data.n_atoms/48, nn_data.order_parameter, 'o-', label=rf"$E_{{nn}} = {nn}$", color=palette[idx])

# Label and style each subplot
for i, ax in enumerate(axes):
    ax.set_ylabel(ylabels[i])
    ax.set_title(titles[i], fontsize=14)
    ax.legend(fontsize=12)
    ax.grid(alpha=0.5)

axes[-1].set_xlabel(r"Coverage $\Theta$")

plt.tight_layout()
plt.show()

In [ ]:
# EXECUTE THIS CELL

def run_simulation(lattice, n_atoms, n_steps, properties_to_collect=['lattice'], data_collection_interval=1):
    """
    Run lattice MC simulation

    Parameters:
    - lattice: (Lattice) 
        Lattice object
    - n_atoms: (int)
        number of atoms to simulate
    - n_steps: (int)
        number of simulation steps
    - properties_to_collect: (list)
        List of properties of Simulation class to store at every simulation step.
    - data_collection_interval: (int)
        Collect data every 'x' number of steps

    Returns:
    - data: (list)
        List of dictionaries with properties (first and last 10 snapshots only).
    """
    s = lattice_mc.Simulation()
    s.lattice = lattice
    s.set_number_of_atoms(n_atoms)
    s.set_nn_energy(4.0 * kT)  # Set E_nn = 4.0
    s.setup_lookup_table()

    def get_properties_dict(properties):
        return {key: copy.deepcopy(getattr(s, key)) for key in properties}

    data = []
    snapshots = []

    for step in range(n_steps):
        s.lattice.jump()
        if (step+1) % data_collection_interval == 0 or step == 0:
            step_data = get_properties_dict(properties_to_collect)
            step_data['step_number'] = step+1
            snapshots.append(step_data)

    # Only keep first and last 10 snapshots
    if len(snapshots) > 20:
        data = snapshots[:10] + snapshots[-10:]
    else:
        data = snapshots

    s.has_run = True
    return data


# === Simulation Setup ===
a, b = 4, 3
spacing = 1.0
lattice = lattice_mc.init_lattice.honeycomb_lattice(a, b, spacing, alternating_sites=True)

# === Run the simulation with 24 atoms, E_nn = 4.0 ===
data = run_simulation(lattice, n_atoms=24, n_steps=10000, data_collection_interval=1000)

# === Visualize lattice snapshots ===
for idx, d in enumerate(data):
    plot_lattice(d['lattice'])

In [ ]:
# ANIMATION UTILITY
def run_simulation(lattice, n_atoms, n_steps, properties_to_collect=['lattice'], data_collection_interval=1):
    """
    Run lattice MC simulation

    Parameters:
    - lattice: (Lattice) 
        Lattice object
    - n_atoms: (int)
        number of atoms to simulate
    - n_steps: (int)
        number of simulation steps
    - properties_to_collect: (list)
        List of properties of Simulation class to store at every simulation step.
    - data_collection_interval: (int)
        Collect data every 'x' number of steps

    Returns:
    - data: (list)
        List of dictionaries with properties (first and last 10 snapshots only).
    """
    s = lattice_mc.Simulation()
    s.lattice = lattice
    s.set_number_of_atoms(n_atoms)
    s.set_nn_energy(4.0 * kT)  # Set E_nn = 4.0
    s.setup_lookup_table()

    def get_properties_dict(properties):
        return {key: copy.deepcopy(getattr(s, key)) for key in properties}

    snapshots = []
    for step in range(n_steps):
        s.lattice.jump()
        if (step + 1) % data_collection_interval == 0 or step == 0:
            step_data = get_properties_dict(properties_to_collect)
            step_data['step_number'] = step + 1
            snapshots.append(step_data)

    # Keep only the first and last 20 snapshots
    if len(snapshots) > 40:
        data = snapshots[:10] + snapshots[-20:]
    else:
        data = snapshots

    s.has_run = True
    return data

# === Animation Utility ===
def animate_lattice_snapshots(data):
    fig, ax = plt.subplots()

    def init():
        ax.clear()
        plot_lattice(data[0]['lattice'], ax=ax)
        ax.set_title(f"Step {data[0]['step_number']}")
        return ax,

    def update(frame):
        ax.clear()
        plot_lattice(data[frame]['lattice'], ax=ax)
        ax.set_title(f"Step {data[frame]['step_number']}")
        return ax,

    ani = FuncAnimation(fig, update, frames=len(data), init_func=init, blit=False, interval=500, repeat=True)
    plt.close(fig)
    return HTML(ani.to_jshtml())

# === Simulation Setup ===
a, b = 4, 3
spacing = 1.0
lattice = lattice_mc.init_lattice.honeycomb_lattice(a, b, spacing, alternating_sites=True)

# === Run the simulation with 24 atoms, E_nn = 4.0 ===
data = run_simulation(lattice, n_atoms=24, n_steps=1000, data_collection_interval=1)

# === Animate lattice snapshots ===
animate_lattice_snapshots(data)

<details>
<summary>Solution</summary>

As the nearest-neighbor repulsion energy increases, $f$ decreases as the lattice occupancy approaches $50\%$. 

The order parameter plot shows that this is associated with a preference for all the mobile particles to order on one sublattice.

The system tends to form an ordered structure (e.g., a checkerboard pattern on a square lattice).

A particle from the full sublattice may jump into the empty one (a disordering jump), but such a move violates the preferred order, so it's energetically discouraged or quickly "undone".

The particle often jumps right back — this is the reordering jump.

*These back-and-forth motions are highly correlated and don't contribute much to net diffusion $\rightarrow$ tracer motion is inefficient $\rightarrow$ $f$ is low.*

This explains the minimum in $f$ at $\Theta = 0.5$: you get lots of futile, correlated moves that don't advance diffusion.


####  What happens above 50% coverage? The "vacant" sublattice gets filled

Now we are inserting extra particles into the "vacant" sublattice: this disrupts the perfect order.

It opens up new paths: now two particles — one on each sublattice — may participate in a kind of pair exchange.

* Imagine particle A hops from the full sublattice to the vacant one.

* Then particle B (already on the vacant sublattice) jumps into A's just-vacated site.

* This pair of hops (interstitialcy-like motion)breaks the ordering locally, but doesn't violate the global structure too much.

* The resulting particle motion is less correlated, i.e., more diffusive, so $f$ increases again.